In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
os.getcwd()

'/kaggle/working'

In [4]:
import multiprocessing

cores_available = multiprocessing.cpu_count()

In [5]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/foursquare-location-matching/sample_submission.csv
/kaggle/input/foursquare-location-matching/pairs.csv
/kaggle/input/foursquare-location-matching/train.csv
/kaggle/input/foursquare-location-matching/test.csv


Split data with knn on lat and long


In [6]:
# pd.read_csv('kaggle/input/foursquare-location-matching/pairs.csv')

In [7]:
# pd.read_csv('kaggle/input/foursquare-location-matching/test.csv')

#  Submission Example:

In [8]:
pd.read_csv('/kaggle/input/foursquare-location-matching/sample_submission.csv')

,id,matches
0,E_00001118ad0191,E_00001118ad0191
1,E_000020eb6fed40,E_000020eb6fed40
2,E_00002f98667edf,E_00002f98667edf
3,E_001b6bad66eb98,E_001b6bad66eb98 E_0283d9f61e569d
4,E_0283d9f61e569d,E_0283d9f61e569d E_001b6bad66eb98


# Read training set:

In [9]:
df = pd.read_csv('/kaggle/input/foursquare-location-matching/train.csv')

In [10]:
df.head(5)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb


In [11]:
df.groupby('country').agg({'id':'count'}).sort_values(by='id',ascending=False).rename(columns={'id':'countries_count'}).describe()

,countries_count
count,221.000000
mean,5152.945701
std,21442.432855
min,1.000000
25%,13.000000
50%,113.000000
75%,1429.000000
max,245284.000000


In [12]:
df.groupby('point_of_interest').agg({'id':'count'}).sort_values(by='id',ascending=False).rename(columns={'id':'POI_count'})

,POI_count
point_of_interest,
P_fb339198a31db3,332
P_3a3fbd26e16269,199
P_f70a0977377171,166
P_9290e406eb58ac,145
P_fbe61733fcdebe,132
...,...
P_6c88990990db38,1
P_6c884940447aa7,1
P_6c8847f66224e2,1


In [13]:
df.loc[df['point_of_interest']=='P_fb339198a31db3']

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
10820,E_02761e40f68230,Sukarno Hatta airport,-7.329110,108.213608,NaN,NaN,NaN,NaN,ID,NaN,NaN,Airport Terminals,P_fb339198a31db3
14686,E_0351377fc6b71d,Bandara Soekarno Hatta,-6.143365,106.729291,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3
19520,E_046132a06d8bc5,Bandar Djakarta,-6.117818,106.855710,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3
21192,E_04c39deab3aba2,"Bandara Soekarnao Hatta...^^,",-6.234635,106.874453,NaN,NaN,NaN,NaN,ID,NaN,NaN,Airport Terminals,P_fb339198a31db3
21433,E_04d15c990bfd9e,Bandara Soekarno Hatta,1.541611,98.915245,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120868,E_fbfa341173cef3,Soekarno Hatta International Airport,-7.952919,112.628568,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3
1133655,E_fed28e5ada5e81,Otw Soetta,-6.261004,106.767409,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3
1137194,E_ffa093e04175aa,Kawasan Soetta,-6.109597,106.653188,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3
1138441,E_ffe8c72a51f9c6,Bandara Soekarno Hatta,-6.130148,106.657134,NaN,Jakarta,NaN,NaN,ID,NaN,NaN,Buildings,P_fb339198a31db3


In [14]:
df.loc[df['point_of_interest']=='P_3a3fbd26e16269']

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
5864,E_01561528896296,Deleted Venue (SEO),53.977069,38.186727,NaN,NaN,NaN,NaN,RU,NaN,NaN,Mattress Stores,P_3a3fbd26e16269
12290,E_02c9ac4b08b798,Deleted Venue (SEO),64.224874,41.657732,NaN,NaN,NaN,NaN,RU,NaN,NaN,Mattress Stores,P_3a3fbd26e16269
26065,E_05d696f881a2ab,Deleted Venue (SEO),59.412537,24.666926,NaN,NaN,NaN,NaN,EE,NaN,NaN,Mattress Stores,P_3a3fbd26e16269
47312,E_0a958576584978,Deleted Venue (SEO),54.315757,48.399682,NaN,NaN,NaN,NaN,RU,NaN,NaN,Mattress Stores,P_3a3fbd26e16269
47616,E_0aa699e0d786d8,Deleted Venue (SEO),58.057184,38.768620,NaN,NaN,NaN,NaN,RU,NaN,NaN,Mattress Stores,P_3a3fbd26e16269
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118135,E_fb5cc62697c52f,Deleted Venue (SEO),55.850651,37.520563,NaN,NaN,NaN,NaN,RU,NaN,NaN,Mattress Stores,P_3a3fbd26e16269
1131160,E_fe46470b6468e1,Deleted Venue (SEO),50.642078,36.572086,NaN,NaN,NaN,NaN,RU,NaN,NaN,Mattress Stores,P_3a3fbd26e16269
1133143,E_feb4399ea9b283,Deleted Venue (SEO),58.377787,26.730695,NaN,NaN,NaN,NaN,EE,NaN,NaN,Mattress Stores,P_3a3fbd26e16269
1135479,E_ff3e99adafc6c0,Deleted Venue (SEO),57.636972,39.862831,NaN,NaN,NaN,NaN,RU,NaN,NaN,Mattress Stores,P_3a3fbd26e16269


In [15]:
df[['lat_rad','long_rad']] = df[['latitude','longitude']].apply(np.deg2rad)

In [16]:
df

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest,lat_rad,long_rad
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e,0.887674,0.063429
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83,-0.399807,-0.753603
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477,0.240521,1.753792
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3,0.660511,0.485973
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb,0.756394,-0.075517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138807,E_ffffb80854f713,青ガエル,35.659020,139.700780,NaN,渋谷区,東京都,150-0043,JP,NaN,NaN,NaN,P_7ccbeab96cd82e,0.622367,2.438239
1138808,E_ffffbf9a83e0ba,Deshon Place,40.872116,-79.945343,325 New Castle Rd,Butler,PA,16001,US,NaN,NaN,Housing Developments,P_db0abc418e7365,0.713353,-1.395309
1138809,E_ffffc572b4d35b,İzmir Adnan Menderes Havaalanı,38.423733,27.142826,NaN,İzmir,NaN,NaN,TR,NaN,NaN,Airport Services,P_ae96252a6a9380,0.670621,0.473732
1138810,E_ffffca745329ed,焼肉 和家,35.710712,139.775000,上野6-13-6,Taitō,東京都,110-0005,JP,NaN,338362989,BBQ Joints,P_146662f246d418,0.623269,2.439534


In [17]:
from sklearn.neighbors import BallTree

In [18]:
ball = BallTree(df[['lat_rad','long_rad']].values, metric='haversine')


In [32]:
from tqdm import tqdm

In [34]:
from joblib import parallel_backend

with parallel_backend('threading', n_jobs=max(cores_available-1,1)):
    bt_distances, bt_indices = tqdm(ball.query(df[['lat_rad','long_rad']].values, k = 10))

100%|██████████| 2/2 [00:00<00:00, 19831.22it/s]


In [36]:
bt_indices, bt_distances

(array([[      0,  704744,  280465, ...,   75629,  324453,  318934],
        [      1,  860109, 1101190, ...,  911520, 1043520,   24873],
        [      2,  812192,  119820, ...,  509371,  176102, 1032718],
        ...,
        [ 580533, 1113074, 1031754, ...,  293310,  284087,   69015],
        [1138810,  135823,  989208, ...,  210362,  806167,   20470],
        [1138811,  670304,  119981, ..., 1023047,  414736,  471379]]),
 array([[0.00000000e+00, 2.10947858e-05, 2.15842285e-05, ...,
         1.10219750e-04, 1.14744388e-04, 1.19106486e-04],
        [0.00000000e+00, 2.04985243e-06, 3.84483818e-06, ...,
         8.50198084e-06, 8.71332491e-06, 9.69600949e-06],
        [0.00000000e+00, 4.61368270e-06, 3.90825291e-05, ...,
         5.27145297e-05, 6.16581774e-05, 6.32165567e-05],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         6.25684539e-08, 6.98131701e-08, 6.98131701e-08],
        [0.00000000e+00, 4.26335927e-07, 4.32964967e-07, ...,
         4.95856

100%|██████████| 2/2 [00:00<00:00, 23431.87it/s]


(array([[      0,  704744,  280465, ...,   75629,  324453,  318934],
        [      1,  860109, 1101190, ...,  911520, 1043520,   24873],
        [      2,  812192,  119820, ...,  509371,  176102, 1032718],
        ...,
        [ 580533, 1113074, 1031754, ...,  293310,  284087,   69015],
        [1138810,  135823,  989208, ...,  210362,  806167,   20470],
        [1138811,  670304,  119981, ..., 1023047,  414736,  471379]]),
 array([[0.00000000e+00, 2.10947858e-05, 2.15842285e-05, ...,
         1.10219750e-04, 1.14744388e-04, 1.19106486e-04],
        [0.00000000e+00, 2.04985243e-06, 3.84483818e-06, ...,
         8.50198084e-06, 8.71332491e-06, 9.69600949e-06],
        [0.00000000e+00, 4.61368270e-06, 3.90825291e-05, ...,
         5.27145297e-05, 6.16581774e-05, 6.32165567e-05],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         6.25684539e-08, 6.98131701e-08, 6.98131701e-08],
        [0.00000000e+00, 4.26335927e-07, 4.32964967e-07, ...,
         4.95856

In [20]:
from scipy import spatial

In [21]:
train_lat = df['latitude'].values
train_long = df['longitude'].values
train_lat_long = tuple(zip(train_lat, train_long))
tree = spatial.KDTree(train_lat_long)
loc_data = tree.query(train_lat_long,350, workers=cores_available)    
distances, indices = loc_data

In [22]:
distances

array([[0.00000000e+00, 1.35112907e-03, 1.38057873e-03, ...,
        1.09507634e-01, 1.09643301e-01, 1.09792356e-01],
       [0.00000000e+00, 1.21718088e-04, 2.20965808e-04, ...,
        5.95484143e-03, 5.95936489e-03, 5.97417529e-03],
       [0.00000000e+00, 2.72053602e-04, 2.28360414e-03, ...,
        9.47766458e-03, 9.49910749e-03, 9.50822065e-03],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        8.05849893e-03, 8.08579624e-03, 8.09036751e-03],
       [0.00000000e+00, 2.70771817e-05, 2.74128695e-05, ...,
        7.86268548e-03, 7.91684185e-03, 7.93465524e-03],
       [0.00000000e+00, 1.35906027e-02, 1.55303092e-02, ...,
        1.31013607e+00, 1.31045139e+00, 1.31047792e+00]])

In [23]:
indices

array([[      0,  280465,  704744, ...,  268153,  503695,  178308],
       [      1,  860109, 1101190, ...,   12277,    2488,  449670],
       [      2,  812192,  119820, ...,  583054,  640379,   70541],
       ...,
       [ 789114, 1113074,  226673, ..., 1109500,  423926,  482033],
       [1138810,  135823,  989208, ...,  582643,  240754,  339359],
       [1138811,  670304,  119981, ...,  885424,  848782,   22744]])

In [24]:
values_count = df.point_of_interest.apply(lambda x: len(x))

In [25]:
df.groupby('point_of_interest').agg({'id':lambda x: ",".join(x)})

,id
point_of_interest,
P_00000458b5ff8e,E_b4a705f3f2e85e
P_00001c309a5e0a,"E_3b8a876a004093,E_7b12ba037d7af8,E_a66af82051..."
P_000047d288cb51,E_9822da0f40eaa3
P_0000561fe92bed,"E_2f12c350e37a69,E_e14714f4aa6461"
P_0000bccb92573c,"E_408896b60c7893,E_822bc2df39ec06"
...,...
P_ffff287983c417,"E_47d063fddd9115,E_ca42d07c618f1c"
P_ffff42b86df943,E_4d52ca0489f774
P_ffff47d56844f9,E_c32f2d8e15a529


In [26]:
from sklearn.neighbors import KNeighborsRegressor
